```
This notebook shows some tests related to passive drifters simulated
on the GPU.

Copyright (C) 2018, 2019  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Testing drift with varying bathymetry

#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import gridspec


import os
import datetime
import sys
from importlib import reload
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../')))

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import CDKLM16, PlotHelper, Common, IPythonMagic
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *
from SWESimulators import GPUDrifterCollection


In [ ]:
#%setup_logging passive_gpu_drifter.log
%cuda_context_handler gpu_ctx

# Passive drift trajectories in the SWE simulators

Here, we will make a naive implementation of particles drifting within our simplified ocean models.
For simplicity, a non-staggered implementation is chosen, as it makes it easier to evaluate the velocity field.


In [ ]:
nx = 50
ny = 50

dx = 4.0
dy = 4.0
dataShape_Hi = ((ny+4+1, nx+4+1))

def runtest(Hi, simulate=True):
    
    figHi = plt.figure(figsize=(5,4))
    plt.imshow(Hi, origin='lower', interpolation='None')
    plt.colorbar()
    
    if not simulate:
        return
    
    #Coriolis well balanced reconstruction scheme
    dt = 0.1
    g = 9.81

    f = 0.0
    r = 0.0

    ghosts = np.array([2,2,2,2]) # north, east, south, west
    validDomain = np.array([2,2,2,2])
    bc = Common.BoundaryConditions(2,2,2,2)

    # Define which cell index which has lower left corner as position (0,0)
    x_zero_ref = 2
    y_zero_ref = 2

    dataShape = (ny + ghosts[0]+ghosts[2], 
                 nx + ghosts[1]+ghosts[3])

    eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
    hu0 = np.zeros(dataShape, dtype=np.float32, order='C');
    hv0 = np.ones(dataShape, dtype=np.float32, order='C')*100;



    #Calculate radius from center of bump for plotting
    y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
    radius = None


    sim = CDKLM16.CDKLM16(gpu_ctx, eta0, hu0, hv0, Hi,
                          nx, ny, dx, dy, dt, g, f, r,
                          boundary_conditions=bc)

    drifters = GPUDrifterCollection.GPUDrifterCollection(gpu_ctx, 64, boundaryConditions=bc,
                                                         domain_size_x=dx*nx,
                                                         domain_size_y=dy*ny)
    sim.attachDrifters(drifters)

    eta1, hu1, hv1 = sim.download()
    fig = plt.figure()
    plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                    eta1, hu1, hv1)

    plotter.showDrifters(sim.drifters)

    T = 100
    sub_t = *dt

    def animate(i):
        if (i>0):
            #t = sim.step(sub_t)
            t = sim.drifterStep(sub_t)
        else:
            t = 0.0

        #eta1, hu1, hv1 = sim.download(interior_domain_only=True)
        #plotter.plot(eta1, hu1, hv1)

        plotter.showDrifters(sim.drifters)

        fig.suptitle("CDKLM16 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

        if (i%20 == 0):
            print( "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax eta: " + str(np.max(eta1)) + \
            "\tMax hu: " + str(np.max(hu1)) + \
            "\tMax hv: " + str(np.max(hv1)))

    anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
    plt.close(anim._fig)
    return anim



In [ ]:
# Testing on varying H
Hi = np.zeros(dataShape_Hi, dtype=np.float32, order='C')
for i in range(dataShape_Hi[1]):
    Hi[:, i] = np.cos(10*i/nx)*5 + 10
    
runtest(Hi, True)

In [ ]:
# Testing on varying H
Hi = np.zeros(dataShape_Hi, dtype=np.float32, order='C')
for j in range(dataShape_Hi[0]):
    Hi[j, :] = np.cos(10*j/ny)*5 + 10
    
runtest(Hi, True)

In [ ]:
# Testing on varying H
Hi = np.zeros(dataShape_Hi, dtype=np.float32, order='C')
for j in range(dataShape_Hi[0]):
    for i in range(dataShape_Hi[1]):
        Hi[j, i] = np.cos(10*j/ny)*np.cos(10*i/nx)*5 + 10
    
runtest(Hi, True)